#Fashion Image Search with Picture Descriptions
                                             -SRIVALLI VANGAVETI

In [49]:
!git clone https://github.com/XiaoxiaoGuo/fashion-iq.git


Cloning into 'fashion-iq'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 140 (delta 18), reused 10 (delta 10), pack-reused 109 (from 1)
Receiving objects: 100% (140/140), 1.48 MiB | 4.72 MiB/s, done.
Resolving deltas: 100% (46/46), done.


Clone the FashionIQ repositories


The FashionIQ dataset contains images and captions for dresses, shirts, and tops.


The metadata repository contains additional information such as image URLs.


In [50]:
!git clone https://github.com/hongwang600/fashion-iq-metadata.git

Cloning into 'fashion-iq-metadata'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17 (from 1)
Receiving objects: 100% (17/17), 1.49 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [51]:
!pip install torch torchvision transformers matplotlib

In [53]:
#Load a sample captions file

# This JSON file contains training captions for dress images
import json
captions_file = "fashion-iq/captions/cap.dress.train.json"

# Open the JSON file and load its content into a Python list
with open(captions_file, "r") as f:
    captions_data = json.load(f)

# Print the first entry
print(json.dumps(captions_data[0], indent=2))


{
  "target": "B008BHCT58",
  "candidate": "B003FGW7MK",
  "captions": [
    "is solid black with no sleeves",
    "is black with straps"
  ]
}


In [54]:
import os
import json
import pandas as pd


#Set up directories for processing

# BASE_DIR: main FashionIQ dataset directory
# CAPTIONS_DIR: directory containing all captions
# IMAGE_SPLITS_DIR: directory containing train/val/test splits
# OUTPUT_DIR: directory where processed CSVs will be saved

BASE_DIR = "fashion-iq"
CAPTIONS_DIR = os.path.join(BASE_DIR, "captions")
IMAGE_SPLITS_DIR = os.path.join(BASE_DIR, "image_splits")
OUTPUT_DIR = os.path.join(BASE_DIR, "annotations")
os.makedirs(OUTPUT_DIR, exist_ok=True)


#Function to load image splits
def load_image_splits(category, split):
    """
    Load image splits for a given category and split.
    """
    split_file = os.path.join(IMAGE_SPLITS_DIR, f"split.{category}.{split}.json")
    with open(split_file, "r") as f:
        return set(json.load(f))


#Function to process a category
def process_category(category):
    """
    Process all splits (train, val, test) for a given category and return combined data.
    """
    splits = ["train", "val", "test"]
    rows = [] # This will store all structured data

    for split in splits:
        # Load valid image IDs for this split
        valid_images = load_image_splits(category, split)

        # Load captions JSON for this category and split
        captions_file = os.path.join(CAPTIONS_DIR, f"cap.{category}.{split}.json")
        if not os.path.exists(captions_file):
            print(f"File not found: {captions_file}")
            continue

        with open(captions_file, "r") as f:
            captions_data = json.load(f)

        for item in captions_data:
            ref_image = item["candidate"] # Reference image ID
            captions = item.get("captions", []) # List of textual descriptions

            target_image = item.get("target", "unknown" if split == "test" else None)

            # Skip images that are not in the valid image split
            if ref_image not in valid_images:
                continue

            if split != "test" and (not target_image or target_image not in valid_images):
                continue

            # For each caption, create a structured row
            for caption in captions:
                rows.append({
                    "split": split,                 # train, val, or test
                    "category": category,           # dress/shirt/toptee
                    "reference_image": ref_image,   # candidate image
                    "target_image": target_image,   # target image
                    "text_description": caption     # caption text
                })

    return rows


#Process all categories and save as CSV
categories = ["dress", "shirt", "toptee"]
for category in categories:
    print(f"Processing category: {category}")
    rows = process_category(category)

    # Convert list of dictionaries into a pandas DataFrame for easier handling
    df_category = pd.DataFrame(rows)

    # Save DataFrame to CSV
    category_csv_path = os.path.join(OUTPUT_DIR, f"{category}_annotations.csv")
    df_category.to_csv(category_csv_path, index=False)
    print(f"CSV file saved for category '{category}' at: {category_csv_path}")


Processing category: dress
CSV file saved for category 'dress' at: fashion-iq/annotations/dress_annotations.csv
Processing category: shirt
CSV file saved for category 'shirt' at: fashion-iq/annotations/shirt_annotations.csv
Processing category: toptee
CSV file saved for category 'toptee' at: fashion-iq/annotations/toptee_annotations.csv


In [55]:
import json

#Inspect first annotation entry for preprocessing
annotation_file = "/content/fashion-iq/captions/cap.dress.train.json"

with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# Print the first entry
print(annotations[0])


{'target': 'B008BHCT58', 'candidate': 'B003FGW7MK', 'captions': ['is solid black with no sleeves', 'is black with straps']}


In [57]:
# Inspect all items (keys and values) in the first annotation entry
annotations[0].items()

dict_items([('target', 'B008BHCT58'), ('candidate', 'B003FGW7MK'), ('captions', ['is solid black with no sleeves', 'is black with straps'])])

In [58]:
target = annotations[0]['target']     # Target image ID
candidate = annotations[0]['candidate']  # Reference image ID
captions = annotations[0]['captions']    # List of captions

# **DATA PREPROCESSING**

In [59]:
import os
import json
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPTokenizer

# Define a custom PyTorch Dataset for FashionIQ

class FashionIQDataset(Dataset):
    def __init__(self, annotation_file, transform=None, tokenizer=None):
        """
        Args:
            annotation_file (str): Path to the JSON annotation file.
            transform (callable, optional): Transformation to apply to images (not used here).
            tokenizer (callable, optional): Tokenizer for captions.
        """

        # Load JSON annotations into memory
        with open(annotation_file, 'r') as f:
            self.annotations = json.load(f)

        # Store optional transform and tokenizer
        self.transform = transform
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.annotations)   # Return the total number of entries in the dataset


        """
        Get a single item from the dataset at index `idx`.
        Returns a dictionary containing:
        - 'reference': candidate image ID
        - 'target': target image ID
        - 'text': tokenized text description (or raw text if tokenizer not provided)
        """

    def __getitem__(self, idx):
        sample = self.annotations[idx]  # Get the annotation at the given index
        reference = sample['candidate']  # Candidate image ID
        target = sample['target']        # Target image ID
        text_description = sample['captions'][0]  # Take the first caption from the list


        # If a tokenizer is provided, convert text description to token tensors
        # Pad the text to a fixed length
        # Maximum sequence length for CLIP
        # Return PyTorch tensor
        if self.tokenizer:
            text_tokens = self.tokenizer(text_description, padding='max_length', max_length=77, return_tensors="pt")
        else:
            text_tokens = text_description             # Otherwise, keep text as raw string
        return {
            "reference": reference,
            "target": target,
            "text": text_tokens
        }

# Initialize the CLIP tokenizer from the pretrained model
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# Path to the annotations JSON file for dresses
ANNOTATION_FILE = "fashion-iq/captions/cap.dress.train.json"

# Create the dataset object
dataset = FashionIQDataset(
    annotation_file=ANNOTATION_FILE,
    transform=None,      # No image transform applied yet
    tokenizer=tokenizer  # Use CLIP tokenizer for text
)

# Create a DataLoader to iterate through dataset in batches
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


# Iterate over the first batch to inspect contents
for i, batch in enumerate(dataloader):
    reference = batch['reference'][0]      # Get first reference image ID in the batch
    target = batch['target'][0]            # Get first target image ID in the batch
    text_description = batch['text']       # Get tokenized text for the batch

    # Print batch info for verification
    print(f"Batch {i + 1}:")
    print(f"  Reference (Candidate): {reference}")
    print(f"  Target: {target}")
    print(f"  Text Description (Raw/Tokenized): {text_description}")

    # Stop after the first batch
    if i == 0:
        break


Batch 1:
  Reference (Candidate): B008VILBSI
  Target: B00AIBYOW6
  Text Description (Raw/Tokenized): {'input_ids': tensor([[[49406,   533,   731,  ..., 49407, 49407, 49407]],

        [[49406,   791,  1538,  ..., 49407, 49407, 49407]],

        [[49406,   533,  1746,  ..., 49407, 49407, 49407]],

        ...,

        [[49406,   533, 18737,  ..., 49407, 49407, 49407]],

        [[49406,  6148,  1449,  ..., 49407, 49407, 49407]],

        [[49406,   533, 20350,  ..., 49407, 49407, 49407]]]), 'attention_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])}


In [60]:
import os
import random
import requests   # To download images from URLs
import json

# Path to ASIN-to-URL mapping file
ASIN2URL_FILE = "fashion-iq-metadata/image_url/asin2url.dress.txt"

# Path to annotation JSON file for dresses
ANNOTATION_FILE = "fashion-iq/captions/cap.dress.train.json"

# Directory where downloaded images will be saved
OUTPUT_DIR = "fashion-iq-metadata/image_url/downloaded_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create directory if it doesn't exist

# Maximum number of images to download
NUM_IMAGES = 1000

# Load annotations JSON
with open(ANNOTATION_FILE, 'r') as f:
    annotations = json.load(f)

# Collect all valid ASINs from candidate and target fields in annotations
valid_asins = {annotation['candidate'] for annotation in annotations} | {annotation['target'] for annotation in annotations}

# Read the ASIN-to-URL file and keep only pairs where ASIN is in valid_asins
with open(ASIN2URL_FILE, 'r') as f:
    # Each line is split into [ASIN, URL]
    asin_url_pairs = [line.strip().split(maxsplit=1) for line in f if line.split(maxsplit=1)[0] in valid_asins]

# Randomly select a subset of ASIN-URL pairs to limit downloads
random_subset = random.sample(asin_url_pairs, min(NUM_IMAGES, len(asin_url_pairs)))

# Download images from the selected subset
for asin, url in random_subset:
    # Set output file path for the image
    output_path = os.path.join(OUTPUT_DIR, f"{asin}.jpg")

    # Only download if the image file doesn't already exist
    if not os.path.exists(output_path):
        try:
            print(f"Downloading {asin} from {url}")

            # Send HTTP GET request to download image
            response = requests.get(url, stream=True)

            # Check if the request was successful
            if response.status_code == 200:
                # Write image in chunks to avoid memory issues
                with open(output_path, 'wb') as img_file:
                    for chunk in response.iter_content(1024):  # 1 KB chunks
                        img_file.write(chunk)
            else:
                print(f"Failed to download {asin}: HTTP {response.status_code}")
        except Exception as e:
            # Catch any errors (network, invalid URL, etc.)
            print(f"Error downloading {asin}: {e}")


Failed to download B00E9OR1YG: HTTP 404
Failed to download B00A0BI83K: HTTP 404
Failed to download B00B81XW5A: HTTP 404
Failed to download B00FNJDPNW: HTTP 404
Failed to download B00361FM12: HTTP 404
Failed to download B00CXMLGT2: HTTP 404
Failed to download B00E7PLDPU: HTTP 404
Failed to download B00AKTD2BU: HTTP 404
Failed to download B00AHGAAUW: HTTP 404
Failed to download B00DI7BZI8: HTTP 404
Failed to download B00DRCJ270: HTTP 404
Failed to download B00DUR5CF8: HTTP 404
Failed to download B00CBDQWM4: HTTP 404
Failed to download B00FB79ZDU: HTTP 404
Failed to download B00BWISZDS: HTTP 404
Failed to download B00CXWW43E: HTTP 404
Failed to download B00E5ASNDW: HTTP 404
Failed to download B00AKW5QKW: HTTP 404
Failed to download B00DM2FZMQ: HTTP 404
Failed to download B008LR38J4: HTTP 404
Failed to download B00D3F802E: HTTP 404
Failed to download B00E9SDZVU: HTTP 404
Failed to download B00EE0Y4AY: HTTP 404
Failed to download B009YK9DK0: HTTP 404
Failed to download B00CV18O18: HTTP 404


In [61]:
import os
import json
import pandas as pd

# Base directory for the FashionIQ dataset
BASE_DIR = "fashion-iq"

# Directory containing caption JSON files
CAPTIONS_DIR = os.path.join(BASE_DIR, "captions")

# Directory containing train/val/test image split JSON files
IMAGE_SPLITS_DIR = os.path.join(BASE_DIR, "image_splits")

# Directory where processed annotation CSVs will be saved
OUTPUT_DIR = os.path.join(BASE_DIR, "annotations")
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create directory if it doesn't exist

def load_image_splits(category, split):
    """
    Load image splits for a given category and split (train/val/test).

    Args:
        category (str): Category name (e.g., 'dress', 'shirt', 'toptee')
        split (str): Split type ('train', 'val', 'test')

    Returns:
        set: Set of image IDs included in this split
    """
    split_file = os.path.join(IMAGE_SPLITS_DIR, f"split.{category}.{split}.json")
    with open(split_file, "r") as f:
        return set(json.load(f))  # Return as a set for faster lookup

def process_category(category):
    """
    Process all splits for a given category and generate structured rows.

    Args:
        category (str): Category name

    Returns:
        list: List of dictionaries with split, category, reference/target images, and captions
    """
    splits = ["train", "val", "test"]  # Define dataset splits
    rows = []  # Initialize list to store processed rows

    for split in splits:

        # Load the valid image IDs for this split
        valid_images = load_image_splits(category, split)

        # Path to the caption JSON file for this category and split
        captions_file = os.path.join(CAPTIONS_DIR, f"cap.{category}.{split}.json")
        if not os.path.exists(captions_file):
            print(f"File not found: {captions_file}")
            continue  # Skip if file does not exist

        # Load caption data from JSON
        with open(captions_file, "r") as f:
            captions_data = json.load(f)

        # Iterate over each annotation in the JSON
        for item in captions_data:
            ref_image = item["candidate"]  # Reference image ASIN
            captions = item.get("captions", [])  # List of captions describing differences
            target_image = item.get("target", "unknown" if split == "test" else None)  # Target image ASIN

            # Skip if reference image is not part of the valid split
            if ref_image not in valid_images:
                continue

            # For non-test splits, skip if target image is missing or invalid
            if split != "test" and (not target_image or target_image not in valid_images):
                continue

            # Create a row for each caption
            for caption in captions:
                rows.append({
                    "split": split,  # Dataset split
                    "category": category,  # Category name
                    "reference_image": ref_image,  # Reference image ASIN
                    "target_image": target_image,  # Target image ASIN
                    "text_description": caption  # Caption text
                })

    return rows  # Return all processed rows for this category

# Process all categories and save as CSV
categories = ["dress", "shirt", "toptee"]
for category in categories:
    print(f"Processing category: {category}")
    rows = process_category(category)

    # Convert list of dictionaries to pandas DataFrame
    df_category = pd.DataFrame(rows)

    # Save DataFrame as CSV for easy use in model training
    category_csv_path = os.path.join(OUTPUT_DIR, f"{category}_annotations.csv")
    df_category.to_csv(category_csv_path, index=False)
    print(f"CSV file saved for category '{category}' at: {category_csv_path}")


Processing category: dress
CSV file saved for category 'dress' at: fashion-iq/annotations/dress_annotations.csv
Processing category: shirt
CSV file saved for category 'shirt' at: fashion-iq/annotations/shirt_annotations.csv
Processing category: toptee
CSV file saved for category 'toptee' at: fashion-iq/annotations/toptee_annotations.csv


In [62]:
import json

# File paths
ANNOTATION_FILE = "fashion-iq/captions/cap.dress.train.json"  # JSON file with captions for dress images
ASIN2URL_FILE = "fashion-iq-metadata/image_url/asin2url.dress.txt"  # Text file mapping ASINs to image URLs
COMMON_IDS_FILE = "/content/fashion-iq/common_ids.json"  # Output JSON file to save filtered annotations

# Load all ASINs that have image URLs
asin_set = set()  # Use a set for fast lookup
with open(ASIN2URL_FILE, 'r') as f:
    for line in f:
        asin, _ = line.strip().split(maxsplit=1)  # Split line into ASIN and URL
        asin_set.add(asin)  # Add ASIN to the set

# Load annotations from the JSON file
with open(ANNOTATION_FILE, 'r') as f:
    annotations = json.load(f)  # List of dictionaries, each with 'candidate', 'target', and 'captions'

# Filter annotations to keep only those where both candidate and target images exist in ASIN set
filtered_annotations = []  # Initialize list to store filtered annotations
for annotation in annotations:
    candidate_id = annotation['candidate']  # Candidate image ASIN
    target_id = annotation['target']        # Target image ASIN

    # Include only if both candidate and target ASINs have images available
    if candidate_id in asin_set and target_id in asin_set:
        filtered_annotations.append(annotation)

# Save the filtered annotations to a new JSON file
with open(COMMON_IDS_FILE, 'w') as f:
    json.dump(filtered_annotations, f, indent=4)  # Pretty-print with indentation

print(f"Filtered annotations saved to {COMMON_IDS_FILE}")  # Confirmation message


Filtered annotations saved to /content/fashion-iq/common_ids.json


CLEANING DOWNLOADED IMAGES SUCH THAT ONLY FOUND ASINS ARE CONSIDERED FOR TRAINING

In [63]:
import os
import json

# Path to the filtered annotations JSON file
common_ids_file = '/content/fashion-iq/common_ids.json'

# Directories to search for downloaded images
image_directories = [
    '/content/fashion-iq',
    '/content/fashion-iq-metadata/image_url/downloaded_images',
    '/content/fashion-iq-metadata/image_url/broken_links'
]

# Directory to save cleaned image info
output_dir = '/content/fashion-iq/images/dress'
asin_to_directory_file = os.path.join(output_dir, 'asin_to_directory.json')

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load the filtered annotations
with open(common_ids_file, 'r') as f:
    common_ids_data = json.load(f)

# Create a set of all candidate and target ASINs
common_ids = {entry['candidate'] for entry in common_ids_data} | {entry['target'] for entry in common_ids_data}

# Initialize sets to track found and missing ASINs
found_asins = set()
missing_asins = set(common_ids)
asin_to_directory = {}

# Iterate through each directory to find images corresponding to the ASINs
for directory in image_directories:
    print(f"Searching in: {directory}")
    if os.path.exists(directory):
        for file in os.listdir(directory):
            file_name, file_ext = os.path.splitext(file)  # Split filename and extension
            if file_name in common_ids:  # Check if file corresponds to a required ASIN
                found_asins.add(file_name)
                missing_asins.discard(file_name)  # Remove found ASIN from missing set
                asin_to_directory[file_name] = directory  # Record the directory of the image

# Print summary statistics
print(f"Total common ASINs: {len(common_ids)}")
print(f"Found ASINs: {len(found_asins)}")
print(f"Missing ASINs: {len(missing_asins)}")

# Paths to save JSON lists of found and missing ASINs
found_asins_file = os.path.join(output_dir, 'found_asins.json')
missing_asins_file = os.path.join(output_dir, 'missing_asins.json')

# Save found ASINs to a JSON file
with open(found_asins_file, 'w') as f:
    json.dump(list(found_asins), f)

# Save missing ASINs to a JSON file
with open(missing_asins_file, 'w') as f:
    json.dump(list(missing_asins), f)

# Save mapping of ASINs to directories
with open(asin_to_directory_file, 'w') as f:
    json.dump(asin_to_directory, f)

# Print confirmation messages
print(f"Found ASINs saved to: {found_asins_file}")
print(f"Missing ASINs saved to: {missing_asins_file}")
print(f"ASIN-to-directory mapping saved to: {asin_to_directory_file}")


Searching in: /content/fashion-iq
Searching in: /content/fashion-iq-metadata/image_url/downloaded_images
Searching in: /content/fashion-iq-metadata/image_url/broken_links
Total common ASINs: 7757
Found ASINs: 952
Missing ASINs: 6805
Found ASINs saved to: /content/fashion-iq/images/dress/found_asins.json
Missing ASINs saved to: /content/fashion-iq/images/dress/missing_asins.json
ASIN-to-directory mapping saved to: /content/fashion-iq/images/dress/asin_to_directory.json


# **DATA** **CLEANING**

In [64]:
import os
import json

# Path to the JSON file containing ASINs of images that were successfully found
found_asins_file = '/content/fashion-iq/images/dress/found_asins.json'

# Directory containing all downloaded images
images_directory = '/content/fashion-iq-metadata/image_url/downloaded_images'

# Load the set of found ASINs from the JSON file
with open(found_asins_file, 'r') as f:
    found_asins = set(json.load(f))

# List all files currently in the images directory
all_files = os.listdir(images_directory)

# Counters for tracking deleted and kept files
deleted_files = 0
kept_files = 0

# Iterate over each file in the directory
for file in all_files:
    file_name, file_ext = os.path.splitext(file)  # Separate the filename from its extension
    if file_name not in found_asins:  # Check if the file is not in the set of valid ASINs
        os.remove(os.path.join(images_directory, file))  # Delete files that are not needed
        print(f"Deleted: {file}")
        deleted_files += 1
    else:
        kept_files += 1  # Count files that are kept

# Print summary statistics after cleaning
print(f"Total files in directory: {len(all_files)}")
print(f"Files kept: {kept_files}")
print(f"Files deleted: {deleted_files}")


Total files in directory: 949
Files kept: 949
Files deleted: 0


# **FINDING COMMON FROM CAP.DRESS.TRAIN AND DOWNLOADED IMAGES**

In [65]:
import os
import json

# Directory where downloaded images are stored
downloaded_images_dir = '/content/fashion-iq-metadata/image_url/downloaded_images'

# List of JSON files containing caption data to filter
json_files = [
    '/content/fashion-iq/captions/cap.dress.train.json'
]

# Output JSON file to store filtered entries
output_file = '/content/fashion-iq/captions/cap.train.json'

# Create a set of ASINs corresponding to images that exist in the downloaded images directory
downloaded_asins = {os.path.splitext(file)[0] for file in os.listdir(downloaded_images_dir)}

# Initialize a list to store entries where both candidate and target images exist
common_entries = []

# Iterate through each JSON file to filter entries
for json_file in json_files:
    with open(json_file, 'r') as f:
        data = json.load(f)
        for entry in data:
            target = entry['target']
            candidate = entry['candidate']
            # Only keep entries where both target and candidate images are present
            if target in downloaded_asins and candidate in downloaded_asins:
                common_entries.append(entry)

# Save the filtered entries to the output JSON file
with open(output_file, 'w') as f:
    json.dump(common_entries, f, indent=4)

# Print summary of filtering results
print(f"Total entries in common set: {len(common_entries)}")
print(f"Filtered entries saved to: {output_file}")


Total entries in common set: 84
Filtered entries saved to: /content/fashion-iq/captions/cap.train.json


In [66]:
import os
import json
import shutil

# Path to the filtered captions JSON file
cap_train_json = '/content/fashion-iq/captions/cap.train.json'

# Directory where all downloaded images are stored
downloaded_images_dir = '/content/fashion-iq-metadata/image_url/downloaded_images'

# Directory where only images referenced in cap.train.json will be copied
output_dir = '/content/fashion-iq-metadata/image_url/downloaded_images_short'

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Load the filtered caption data
with open(cap_train_json, 'r') as f:
    cap_train_data = json.load(f)

# Collect all ASINs (both candidate and target) from the filtered captions
cap_train_asins = set()
for entry in cap_train_data:
    cap_train_asins.add(entry['candidate'])
    cap_train_asins.add(entry['target'])

# Copy images corresponding to the collected ASINs to the output directory
copied_count = 0
for asin in cap_train_asins:
    image_file = f"{asin}.jpg"  # Construct the image file name
    src_path = os.path.join(downloaded_images_dir, image_file)  # Source path
    dst_path = os.path.join(output_dir, image_file)  # Destination path
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)
        copied_count += 1
        print(f"Copied: {image_file}")
    else:
        print(f"Image not found: {image_file}")

# Print summary of the copying process
print(f"Total images in cap.train.json: {len(cap_train_asins)}")
print(f"Total images copied: {copied_count}")
print(f"Images saved to: {output_dir}")


Copied: B00BJMIKPK.jpg
Copied: B00FTGZDXY.jpg
Copied: B0060OU0P0.jpg
Copied: B00B7DZNG0.jpg
Copied: B00C4OMRUQ.jpg
Copied: B00A3USRAW.jpg
Copied: B00E0CYF9G.jpg
Copied: B004ZWGGDY.jpg
Copied: B00AIBYOW6.jpg
Copied: B00CFU74SY.jpg
Copied: B008VIU2V0.jpg
Copied: B00DP4WHK4.jpg
Copied: B005TD07GK.jpg
Copied: B004QTQ8FW.jpg
Copied: B00DH8YRWO.jpg
Copied: B009LJ95N4.jpg
Copied: B008FWZP5K.jpg
Copied: B003IF4HLC.jpg
Copied: B008P5OFHQ.jpg
Copied: B00BM1QDWU.jpg
Copied: B00EW41B30.jpg
Copied: B007G10BPA.jpg
Copied: B005KQWIYK.jpg
Copied: B006BBKVPG.jpg
Copied: B0098MOTAI.jpg
Copied: B005EVNJ1C.jpg
Copied: B00BEMDD0C.jpg
Copied: B00CQSTQCW.jpg
Copied: B004LRBIJA.jpg
Copied: B00CDAL83I.jpg
Copied: B006VA9B08.jpg
Copied: B0061K4F7M.jpg
Copied: B006H2GU0E.jpg
Copied: B00BBQCWY4.jpg
Copied: B00ARHNXWI.jpg
Copied: B004SLT8MI.jpg
Copied: B00BM7318S.jpg
Copied: B008UAEXZK.jpg
Copied: B006WXNZHO.jpg
Copied: B00DJZPTO0.jpg
Copied: B0056F24M6.jpg
Copied: B00G7H39YY.jpg
Copied: B008BA7THS.jpg
Copied: B00

In [67]:
import json
import random
import os

# Path to the filtered captions JSON file
input_json = '/content/fashion-iq/captions/cap.train.json'

# Paths where the train and test split JSON files will be saved
train_output_json = '/content/fashion-iq/captions/cap.train_split.json'
test_output_json = '/content/fashion-iq/captions/cap.test_split.json'

# Load the filtered caption data
with open(input_json, 'r') as f:
    data = json.load(f)

# Shuffle the data randomly to ensure a random distribution in train/test sets
random.shuffle(data)

# Compute index for 80-20 split
split_idx = int(len(data) * 0.8)

# Split data into training (80%) and testing (20%) sets
train_data = data[:split_idx]
test_data = data[split_idx:]

# Save training data to JSON file with pretty formatting
with open(train_output_json, 'w') as f:
    json.dump(train_data, f, indent=4)

# Save testing data to JSON file with pretty formatting
with open(test_output_json, 'w') as f:
    json.dump(test_data, f, indent=4)

# Print summary of saved files
print(f"Training data saved to: {train_output_json}")
print(f"Testing data saved to: {test_output_json}")


Training data saved to: /content/fashion-iq/captions/cap.train_split.json
Testing data saved to: /content/fashion-iq/captions/cap.test_split.json


# **OPENCLIP MODEL**

In [68]:
!pip install open_clip_torch

In [69]:
pip install open_clip_torch

In [70]:
import os  # Import os module for file and directory operations
import json  # Import json module to read/write JSON files
import torch  # Import PyTorch for deep learning
from torch.utils.data import Dataset, DataLoader  # Import Dataset and DataLoader for handling data
from PIL import Image  # Import PIL for image loading
from torchvision import transforms  # Import torchvision transforms for image preprocessing
from tqdm import tqdm  # Import tqdm for progress bars
import open_clip  # Import OpenCLIP library
import random  # Import random module for random selection

train_json = '/content/fashion-iq/captions/cap.train_split.json'  # Path to training JSON
images_dir = '/content/fashion-iq-metadata/image_url/downloaded_images_short'  # Directory with images
output_model_path = '/content/fashion-iq/OUTPUT/openclip_model.pt'  # Path to save trained model

batch_size = 32  # Number of samples per batch
num_epochs = 10  # Number of training epochs
learning_rate = 1e-4  # Learning rate for optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available, else CPU

class CaptionDataset(Dataset):
    def __init__(self, json_path, images_dir, transform=None):
        self.data = json.load(open(json_path, 'r'))  # Load JSON annotations
        self.images_dir = images_dir  # Store images directory path
        self.transform = transform  # Store image transform function

    def __len__(self):
        return len(self.data)  # Return number of samples in dataset

    def __getitem__(self, idx):
        entry = self.data[idx]  # Get data entry at index
        candidate_image_path = os.path.join(self.images_dir, f"{entry['candidate']}.jpg")  # Path to candidate image
        text = entry['captions'][0]  # Get first caption text

        if not os.path.exists(candidate_image_path):  # Check if image file exists
            raise FileNotFoundError(f"Image not found: {candidate_image_path}")  # Raise error if missing

        candidate_image = Image.open(candidate_image_path).convert("RGB")  # Open image and convert to RGB
        if self.transform:  # Apply transforms if provided
            candidate_image = self.transform(candidate_image)

        return candidate_image, text  # Return image and text

transform = transforms.Compose([  # Compose multiple image transformations
    transforms.Resize((224, 224)),  # Resize image to 224x224
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize image
])

train_dataset = CaptionDataset(train_json, images_dir, transform=transform)  # Create dataset object
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Create DataLoader for batching

model = open_clip.create_model("ViT-B-32", pretrained="openai")  # Load OpenCLIP ViT-B-32 model
model = model.to(device)  # Move model to GPU or CPU
tokenizer = open_clip.get_tokenizer("ViT-B-32")  # Load tokenizer for text captions

from open_clip import image_transform  # Import image_transform utility
preprocess = image_transform(model.visual.image_size, is_train=False)  # Create preprocessing function for single images

criterion = torch.nn.CrossEntropyLoss()  # Define cross-entropy loss
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)  # Define AdamW optimizer

print("Starting Training...")  # Print message indicating training start
for epoch in range(num_epochs):  # Loop over each epoch
    model.train()  # Set model to training mode
    total_loss = 0.0  # Initialize loss for the epoch
    for images, texts in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):  # Loop over batches
        images = images.to(device)  # Move images to GPU/CPU
        tokenized_texts = tokenizer(texts).to(device)  # Tokenize captions and move to GPU/CPU

        image_features = model.encode_image(images)  # Encode images into feature vectors
        text_features = model.encode_text(tokenized_texts)  # Encode texts into feature vectors

        logits_per_image = image_features @ text_features.T  # Compute similarity logits (image->text)
        logits_per_text = text_features @ image_features.T  # Compute similarity logits (text->image)
        labels = torch.arange(len(images)).to(device)  # Create target labels for loss

        loss = (criterion(logits_per_image, labels) + criterion(logits_per_text, labels)) / 2  # Compute average loss
        total_loss += loss.item()  # Add batch loss to total

        optimizer.zero_grad()  # Reset gradients
        loss.backward()  # Backpropagate
        optimizer.step()  # Update model parameters

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")  # Print epoch loss

os.makedirs(os.path.dirname(output_model_path), exist_ok=True)  # Make directory if it doesn't exist
torch.save(model, output_model_path)  # Save trained model
print(f"Model saved to {output_model_path}")  # Print confirmation

def predict_description(candidate_image_path, model, tokenizer, transform):
    model.eval()  # Set model to evaluation mode
    candidate_image = Image.open(candidate_image_path).convert("RGB")  # Load image
    candidate_image = transform(candidate_image).unsqueeze(0).to(device)  # Transform and add batch dimension

    with torch.no_grad():  # Disable gradient computation
        image_features = model.encode_image(candidate_image)  # Encode image

    return image_features  # Return embedding

existing_images = os.listdir(images_dir)  # List all images in directory
if len(existing_images) == 0:  # Check if directory is empty
    raise FileNotFoundError(f"No images found in {images_dir}")  # Raise error if empty

candidate_image = os.path.join(images_dir, random.choice(existing_images))  # Pick a random image
print("Testing embedding for image:", candidate_image)  # Print image path

embedding = predict_description(candidate_image, model, tokenizer, preprocess)  # Get embedding
print("Predicted embedding shape:", embedding.shape)  # Print embedding shape


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Starting Training...


Epoch 1/10: 100%|██████████| 3/3 [00:01<00:00,  1.97it/s]


Epoch 1/10, Loss: 4.065611998240153


Epoch 2/10: 100%|██████████| 3/3 [00:01<00:00,  2.03it/s]


...

Epoch 8/10, Loss: 2.679178555806478


Epoch 9/10: 100%|██████████| 3/3 [00:02<00:00,  1.09it/s]


Epoch 9/10, Loss: 2.6772395769755044


Epoch 10/10: 100%|██████████| 3/3 [00:01<00:00,  2.16it/s]


Epoch 10/10, Loss: 2.6758139928181968
Model saved to /content/fashion-iq/OUTPUT/openclip_model.pt
Testing embedding for image: /content/fashion-iq-metadata/image_url/downloaded_images_short/B008EQ06I8.jpg
Predicted embedding shape: torch.Size([1, 512])


In [71]:
import os  # Import os module for file and directory operations
from PIL import Image  # Import PIL for opening and saving images
from torchvision import transforms  # Import torchvision transforms for preprocessing
from tqdm import tqdm  # Import tqdm for progress bars

input_dir = '/content/fashion-iq-metadata/image_url/downloaded_images_short'  # Directory with raw images
output_dir = '/content/fashion-iq-metadata/image_url/processed_images'  # Directory to save processed images

os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize pixel values
])

print("Processing images...")  # Inform user that processing has started
for filename in tqdm(os.listdir(input_dir)):  # Loop over all files in input directory with progress bar
    input_path = os.path.join(input_dir, filename)  # Full path to the input image
    output_path = os.path.join(output_dir, filename)  # Full path for saving processed image

    try:
        image = Image.open(input_path).convert("RGB")  # Open image and convert to RGB
        processed_image = transform(image)  # Apply transformations to the image

        processed_image_pil = transforms.ToPILImage()(processed_image)  # Convert tensor back to PIL image
        processed_image_pil.save(output_path, format="JPEG")  # Save processed image in JPEG format
    except Exception as e:  # Handle errors if image cannot be processed
        print(f"Error processing image {filename}: {e}")  # Print error message

print("All images processed and saved to:", output_dir)  # Inform user that processing is complete


Processing images...


100%|██████████| 158/158 [00:01<00:00, 136.69it/s]

All images processed and saved to: /content/fashion-iq-metadata/image_url/processed_images


In [72]:
# Full working script: safely create/load an OpenCLIP model, ensure a saved checkpoint exists,
# then run predict_description on one image. This fixes the FileNotFoundError by creating
# and saving a checkpoint if it doesn't already exist.

import os
import json
import torch
from PIL import Image
import open_clip

# ---------- Settings ----------
MODEL_PATH = "model_complete.pth"   # file that your original code expected
CLIP_ARCH = "ViT-B-32"              # architecture used in your notebook
DEVICE = "cpu"                      # keep CPU-compatible; change to "cuda" if you have GPU
# ------------------------------

device = torch.device(DEVICE)

# Create model and preprocessing transforms (this returns the architecture and the matching preprocess)
# If pretrained='openai' this will load the official pretrained weights.
print("Creating model architecture and transforms...")
model, _, preprocess = open_clip.create_model_and_transforms(CLIP_ARCH, pretrained="openai")
model.to(device)
model.eval()

# If the expected checkpoint exists, try to load it as a state_dict.
# If it does not exist, save the current (pretrained) model's state_dict to MODEL_PATH so subsequent loads succeed.
if os.path.exists(MODEL_PATH):
    try:
        print(f"Found checkpoint at '{MODEL_PATH}'. Loading state_dict...")
        state = torch.load(MODEL_PATH, map_location=device)
        # If the saved object is a full model, it may be necessary to detect that.
        # We'll try to load as state_dict first; if that fails, attempt direct torch.load model.
        if isinstance(state, dict):
            # Common case: saved state_dict
            model.load_state_dict(state)
            print("Loaded state_dict into model.")
        else:
            # If someone saved the full model object (less common/less recommended),
            # replace the model variable with the loaded object.
            model = state.to(device)
            model.eval()
            print("Loaded full model object from checkpoint.")
    except Exception as e:
        # Fall back: warn and continue with the pretrained model in memory
        print("Warning: failed to load checkpoint as state_dict or model object.")
        print("Error:", e)
        print("Continuing with the freshly created pretrained model (not loaded from checkpoint).")
else:
    # No checkpoint file found — create one by saving the current pretrained model's state_dict.
    try:
        print(f"No checkpoint found at '{MODEL_PATH}'. Saving current pretrained model.state_dict() to this path...")
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"Saved pretrained model.state_dict() -> '{MODEL_PATH}'.")
    except Exception as e:
        print("Error when attempting to save the model state_dict:", e)
        print("You may need write permissions for the current working directory.")

# Get tokenizer (OpenCLIP's tokenizer)
tokenizer = open_clip.get_tokenizer(CLIP_ARCH)

# Prediction helper
def predict_description(image_path, model, tokenizer, all_descriptions, preprocess, device="cpu"):
    """
    Given an image_path and a list of candidate descriptions (strings),
    return the description that has the highest cosine similarity with the image embedding.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Open and preprocess image
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to(device)  # shape: (1, C, H, W)

    # Tokenize all descriptions (this returns a tensor of token ids ready for model.encode_text)
    text_tokens = tokenizer(all_descriptions).to(device)

    model.to(device)
    model.eval()
    with torch.no_grad():
        # encode_image returns (batch, dim)
        image_features = model.encode_image(image_input)         # (1, D)
        image_features = torch.nn.functional.normalize(image_features, p=2, dim=-1)

        # encode_text accepts tokenized descriptions (N, token_len)
        text_features = model.encode_text(text_tokens)          # (N, D)
        text_features = torch.nn.functional.normalize(text_features, p=2, dim=-1)

        # similarity: (1, N)
        similarities = (image_features @ text_features.T)

    best_idx = torch.argmax(similarities, dim=1).item()
    return all_descriptions[best_idx]

# -------------------------
# Load candidate descriptions (test captions)
# -------------------------
descriptions_file = "/content/fashion-iq/captions/cap.test_split.json"

if not os.path.exists(descriptions_file):
    # Try alternative paths if the dataset was cloned elsewhere (give a helpful message)
    print("Warning: descriptions file not found at the expected path:", descriptions_file)
    print("Make sure you have cloned the 'fashion-iq' dataset and that the path is correct.")
    # As a fallback, create a tiny dummy list so the demo still runs (but results won't be meaningful).
    all_descriptions = [
        "sleeveless red dress with flared skirt",
        "short-sleeve floral summer dress",
        "long sleeve black top with buttons",
        "blue denim jacket with pockets",
    ]
    print("Using a small fallback list of descriptions (demo only).")
else:
    with open(descriptions_file, "r") as f:
        data = json.load(f)
    # Each entry in FashionIQ captions usually has a 'captions' list
    all_descriptions = [entry["captions"][0] if isinstance(entry.get("captions"), list) and entry["captions"] else "" for entry in data]
    # Filter out any empty description entries
    all_descriptions = [d for d in all_descriptions if d]

# -------------------------
# Run prediction on a sample image
# -------------------------
sample_image = "/content/fashion-iq-metadata/image_url/downloaded_images/B00CL0KUU2.jpg"

if not os.path.exists(sample_image):
    print("Warning: sample image not found at the expected path:", sample_image)
    print("Make sure you have downloaded images to '/content/fashion-iq-metadata/...'.")
    # Optionally you can set sample_image to a different local path here if you have one.
else:
    try:
        predicted_description = predict_description(sample_image, model, tokenizer, all_descriptions, preprocess, device=DEVICE)
        print(f"Input Image: {sample_image}")
        print(f"Predicted Description: {predicted_description}")
    except Exception as e:
        print("Error during prediction:", e)


Creating model architecture and transforms...
Found checkpoint at 'model_complete.pth'. Loading state_dict...
Loaded state_dict into model.
Make sure you have downloaded images to '/content/fashion-iq-metadata/...'.


In [73]:
pip install open_clip_torch

# TRAINING


In [74]:
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import open_clip
import random
import numpy as np

# ============================================
# IMPROVED TRAINING CONFIGURATION
# ============================================

train_json = '/content/fashion-iq/captions/cap.train_split.json'
images_dir = '/content/fashion-iq-metadata/image_url/downloaded_images_short'
output_model_path = '/content/fashion-iq/OUTPUT/openclip_model.pt'

os.makedirs('/content/fashion-iq/OUTPUT', exist_ok=True)

# IMPROVED HYPERPARAMETERS
batch_size = 8  # Smaller batch for better gradient updates with small dataset
num_epochs = 50  # More epochs
learning_rate = 5e-6  # Fine-tuned learning rate
warmup_epochs = 5
weight_decay = 0.1
temperature = 0.07  # Temperature for contrastive loss

device = "cuda" if torch.cuda.is_available() else "cpu"

print("=" * 60)
print("IMPROVED TRAINING CONFIGURATION")
print("=" * 60)
print(f"Device: {device}")
print(f"Batch size: {batch_size}")
print(f"Epochs: {num_epochs}")
print(f"Learning rate: {learning_rate}")
print(f"Warmup epochs: {warmup_epochs}")
print(f"Temperature: {temperature}")
print("=" * 60)

# ============================================
# IMPROVED DATASET WITH DATA AUGMENTATION
# ============================================

class ImprovedFashionDataset(Dataset):
    """
    Improved dataset that:
    1. Uses BOTH captions per sample (doubles effective data)
    2. Applies data augmentation
    3. Includes target images for triplet-style learning
    """
    def __init__(self, json_path, images_dir, preprocess, augment=True):
        with open(json_path, 'r') as f:
            self.raw_data = json.load(f)

        self.images_dir = images_dir
        self.preprocess = preprocess
        self.augment = augment

        # Expand data to use both captions
        self.data = []
        for entry in self.raw_data:
            candidate_path = os.path.join(images_dir, f"{entry['candidate']}.jpg")
            target_path = os.path.join(images_dir, f"{entry['target']}.jpg")

            # Check if both images exist
            if os.path.exists(candidate_path) and os.path.exists(target_path):
                # Add entry for each caption (doubles the data)
                for caption in entry.get('captions', []):
                    if caption:
                        self.data.append({
                            'candidate_path': candidate_path,
                            'target_path': target_path,
                            'caption': caption,
                            'candidate_id': entry['candidate'],
                            'target_id': entry['target']
                        })

        print(f"Loaded {len(self.data)} training samples (expanded from {len(self.raw_data)} entries)")

        # Augmentation transforms
        self.augment_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]

        # Load candidate (reference) image
        candidate_image = Image.open(entry['candidate_path']).convert("RGB")

        # Load target image
        target_image = Image.open(entry['target_path']).convert("RGB")

        # Apply augmentation during training
        if self.augment and random.random() > 0.5:
            candidate_image = self.augment_transform(candidate_image)
            target_image = self.augment_transform(target_image)

        # Apply CLIP preprocessing
        candidate_tensor = self.preprocess(candidate_image)
        target_tensor = self.preprocess(target_image)

        return {
            'candidate': candidate_tensor,
            'target': target_tensor,
            'caption': entry['caption']
        }


# ============================================
# IMPROVED LOSS FUNCTION
# ============================================

class ImprovedContrastiveLoss(nn.Module):
    """
    Improved loss that considers:
    1. Reference image + text should match target image
    2. Standard CLIP contrastive loss
    3. Temperature scaling for better gradients
    """
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, image_features, text_features, target_features=None):
        # Normalize features
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        # Standard CLIP loss (image-text alignment)
        logits_per_image = (image_features @ text_features.T) / self.temperature
        logits_per_text = (text_features @ image_features.T) / self.temperature

        batch_size = image_features.shape[0]
        labels = torch.arange(batch_size, device=image_features.device)

        loss_i2t = self.cross_entropy(logits_per_image, labels)
        loss_t2i = self.cross_entropy(logits_per_text, labels)

        clip_loss = (loss_i2t + loss_t2i) / 2

        # If target features provided, add composed retrieval loss
        if target_features is not None:
            target_features = target_features / target_features.norm(dim=-1, keepdim=True)

            # Combined query (reference image + text) should match target
            combined_query = image_features + text_features
            combined_query = combined_query / combined_query.norm(dim=-1, keepdim=True)

            # Similarity between combined query and target
            logits_composed = (combined_query @ target_features.T) / self.temperature

            composed_loss = self.cross_entropy(logits_composed, labels)

            total_loss = clip_loss + composed_loss
        else:
            total_loss = clip_loss

        return total_loss


# ============================================
# MODEL SETUP
# ============================================

print("\nLoading OpenCLIP model...")
model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
model = model.to(device)
tokenizer = open_clip.get_tokenizer("ViT-B-32")
print("✓ Model loaded successfully")

# ============================================
# DATA PREPARATION
# ============================================

train_dataset = ImprovedFashionDataset(
    train_json,
    images_dir,
    preprocess,
    augment=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True if device == "cuda" else False,
    drop_last=True  # Important for consistent batch sizes
)

print(f"\nDataset size: {len(train_dataset)} samples")
print(f"Number of batches: {len(train_loader)}")

# ============================================
# OPTIMIZER WITH LAYER-WISE LEARNING RATES
# ============================================

# Different learning rates for different parts of the model
# Lower LR for pretrained layers, higher for later layers
def get_parameter_groups(model, base_lr):
    """Create parameter groups with layer-wise learning rate decay."""

    # Visual encoder parameters
    visual_params = []
    # Text encoder parameters
    text_params = []
    # Other parameters
    other_params = []

    for name, param in model.named_parameters():
        if 'visual' in name:
            visual_params.append(param)
        elif 'transformer' in name or 'token_embedding' in name or 'positional_embedding' in name:
            text_params.append(param)
        else:
            other_params.append(param)

    return [
        {'params': visual_params, 'lr': base_lr * 0.1},  # Lower LR for visual
        {'params': text_params, 'lr': base_lr * 0.1},    # Lower LR for text
        {'params': other_params, 'lr': base_lr}          # Normal LR for projection heads
    ]

param_groups = get_parameter_groups(model, learning_rate)
optimizer = torch.optim.AdamW(param_groups, lr=learning_rate, weight_decay=weight_decay)

# Learning rate scheduler with warmup
def get_lr_scheduler(optimizer, num_epochs, warmup_epochs, num_batches):
    total_steps = num_epochs * num_batches
    warmup_steps = warmup_epochs * num_batches

    def lr_lambda(step):
        if step < warmup_steps:
            return step / warmup_steps
        else:
            progress = (step - warmup_steps) / (total_steps - warmup_steps)
            return 0.5 * (1 + np.cos(np.pi * progress))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, num_epochs, warmup_epochs, len(train_loader))

# Loss function
criterion = ImprovedContrastiveLoss(temperature=temperature)

# ============================================
# TRAINING LOOP
# ============================================

print("\n" + "=" * 60)
print("STARTING IMPROVED TRAINING")
print("=" * 60)

best_loss = float('inf')
training_history = []

# Enable gradient checkpointing to save memory
if hasattr(model.visual, 'set_grad_checkpointing'):
    model.visual.set_grad_checkpointing(True)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    num_batches = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")

    for batch in pbar:
        candidate_images = batch['candidate'].to(device)
        target_images = batch['target'].to(device)
        captions = batch['caption']

        # Tokenize captions
        text_tokens = tokenizer(captions).to(device)

        # Forward pass
        candidate_features = model.encode_image(candidate_images)
        target_features = model.encode_image(target_images)
        text_features = model.encode_text(text_tokens)

        # Calculate loss with target features
        loss = criterion(candidate_features, text_features, target_features)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        num_batches += 1

        # Update progress bar
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'lr': f'{current_lr:.2e}'
        })

    # Calculate epoch metrics
    avg_loss = total_loss / num_batches
    current_lr = optimizer.param_groups[0]['lr']

    training_history.append({
        'epoch': epoch + 1,
        'loss': avg_loss,
        'learning_rate': current_lr
    })

    print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {avg_loss:.4f} | LR: {current_lr:.2e}")

    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), output_model_path)
        print(f"  ✓ New best model saved (loss: {avg_loss:.4f})")

    # Early stopping check (optional)
    if avg_loss < 0.1:
        print("  ✓ Loss threshold reached, stopping early")
        break

# ============================================
# SAVE FINAL RESULTS
# ============================================

history_path = '/content/fashion-iq/OUTPUT/training_history.json'
with open(history_path, 'w') as f:
    json.dump(training_history, f, indent=2)

print("\n" + "=" * 60)
print("TRAINING COMPLETED!")
print("=" * 60)
print(f"✓ Best loss: {best_loss:.4f}")
print(f"✓ Model saved to: {output_model_path}")
print(f"✓ History saved to: {history_path}")
print("=" * 60)

IMPROVED TRAINING CONFIGURATION
Device: cuda
Batch size: 8
Epochs: 50
Learning rate: 5e-06
Warmup epochs: 5
Temperature: 0.07

Loading OpenCLIP model...
✓ Model loaded successfully
Loaded 134 training samples (expanded from 67 entries)

Dataset size: 134 samples
Number of batches: 16

STARTING IMPROVED TRAINING


Epoch 1/50: 100%|██████████| 16/16 [00:10<00:00,  1.53it/s, loss=3.5919, lr=1.00e-07]


Epoch 1/50 | Loss: 3.7490 | LR: 1.00e-07
  ✓ New best model saved (loss: 3.7490)


Epoch 2/50: 100%|██████████| 16/16 [00:07<00:00,  2.08it/s, loss=3.5407, lr=2.00e-07]


Epoch 2/50 | Loss: 3.6770 | LR: 2.00e-07
  ✓ New best model saved (loss: 3.6770)


...

Epoch 48/50 | Loss: 0.7281 | LR: 2.43e-09


Epoch 49/50: 100%|██████████| 16/16 [00:05<00:00,  2.93it/s, loss=0.5446, lr=6.09e-10]


Epoch 49/50 | Loss: 0.7638 | LR: 6.09e-10


Epoch 50/50: 100%|██████████| 16/16 [00:05<00:00,  2.68it/s, loss=0.4681, lr=0.00e+00]

Epoch 50/50 | Loss: 0.7125 | LR: 0.00e+00

TRAINING COMPLETED!
✓ Best loss: 0.6920
✓ Model saved to: /content/fashion-iq/OUTPUT/openclip_model.pt
✓ History saved to: /content/fashion-iq/OUTPUT/training_history.json


#Generate Embeddings


In [75]:
import os
import json
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import pickle
import open_clip

# ============================================
# REGENERATE EMBEDDINGS WITH IMPROVED MODEL
# ============================================

print("=" * 60)
print("REGENERATING EMBEDDINGS WITH IMPROVED MODEL")
print("=" * 60)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}\n")

# Paths
IMAGES_DIR = '/content/fashion-iq-metadata/image_url/downloaded_images_short'
MODEL_PATH = '/content/fashion-iq/OUTPUT/openclip_model.pt'
OUTPUT_EMBEDDINGS_FILE = '/content/fashion-iq/OUTPUT/image_embeddings.pkl'
OUTPUT_METADATA_FILE = '/content/fashion-iq/OUTPUT/image_metadata.json'

# Verify model exists
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model not found at {MODEL_PATH}")

os.makedirs('/content/fashion-iq/OUTPUT', exist_ok=True)

# ============================================
# LOAD IMPROVED MODEL
# ============================================

print("Loading improved model...")
model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
model.to(device)

# Load fine-tuned weights
state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict, strict=True)
model.eval()
print("✓ Improved model loaded successfully\n")

# ============================================
# GENERATE EMBEDDINGS
# ============================================

all_embeddings = []
all_image_ids = []
all_image_paths = []

image_files = [f for f in os.listdir(IMAGES_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"Processing {len(image_files)} images...")

with torch.no_grad():
    for filename in tqdm(image_files, desc="Generating embeddings"):
        image_path = os.path.join(IMAGES_DIR, filename)
        image_id = os.path.splitext(filename)[0]

        try:
            image = Image.open(image_path).convert("RGB")
            image_tensor = preprocess(image).unsqueeze(0).to(device)

            embedding = model.encode_image(image_tensor)
            embedding = embedding / embedding.norm(dim=-1, keepdim=True)

            all_embeddings.append(embedding.cpu().numpy())
            all_image_ids.append(image_id)
            all_image_paths.append(image_path)

        except Exception as e:
            print(f"\n⚠️ Error processing {filename}: {e}")
            continue

# Convert to array
all_embeddings = np.vstack(all_embeddings)

print(f"\n✓ Generated {len(all_embeddings)} embeddings")
print(f"✓ Embedding shape: {all_embeddings.shape}")

# Save embeddings
print(f"\nSaving embeddings...")
with open(OUTPUT_EMBEDDINGS_FILE, 'wb') as f:
    pickle.dump({
        'embeddings': all_embeddings,
        'image_ids': all_image_ids,
        'image_paths': all_image_paths
    }, f)

# Save metadata
metadata = {
    'total_images': len(all_image_ids),
    'embedding_dimension': all_embeddings.shape[1],
    'image_ids': all_image_ids,
    'image_paths': all_image_paths
}

with open(OUTPUT_METADATA_FILE, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Embeddings saved to: {OUTPUT_EMBEDDINGS_FILE}")
print(f"✓ Metadata saved to: {OUTPUT_METADATA_FILE}")
print("=" * 60)

REGENERATING EMBEDDINGS WITH IMPROVED MODEL
Using device: cuda

Loading improved model...
✓ Improved model loaded successfully

Processing 158 images...


Generating embeddings: 100%|██████████| 158/158 [00:03<00:00, 41.53it/s]


✓ Generated 158 embeddings
✓ Embedding shape: (158, 512)

Saving embeddings...
✓ Embeddings saved to: /content/fashion-iq/OUTPUT/image_embeddings.pkl
✓ Metadata saved to: /content/fashion-iq/OUTPUT/image_metadata.json


#Image Retrieval Function


In [76]:
import numpy as np
from PIL import Image
import torch
import pickle
import open_clip

# ============================================
# IMPROVED IMAGE + TEXT RETRIEVAL
# ============================================

def retrieve_similar_images_improved(
    query_image_path,
    text_query,
    model,
    preprocess,
    tokenizer,
    device,
    top_k=5,
    text_weight=0.5
):
    """
    Improved retrieval with better similarity computation.
    Uses additive composition as trained.
    """

    # Load embeddings database
    with open('/content/fashion-iq/OUTPUT/image_embeddings.pkl', 'rb') as f:
        data = pickle.load(f)
        all_embeddings = data['embeddings']
        all_image_ids = data['image_ids']
        all_image_paths = data['image_paths']

    model.eval()
    with torch.no_grad():
        # Encode reference image
        query_image = Image.open(query_image_path).convert("RGB")
        query_tensor = preprocess(query_image).unsqueeze(0).to(device)
        image_embedding = model.encode_image(query_tensor)
        image_embedding = image_embedding / image_embedding.norm(dim=-1, keepdim=True)

        # Encode text modification
        if text_query and text_query.strip():
            text_tokens = tokenizer([text_query]).to(device)
            text_embedding = model.encode_text(text_tokens)
            text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)

            # ADDITIVE COMPOSITION (matching training)
            # This is how the model was trained - add image and text features
            combined_embedding = image_embedding + text_embedding
            combined_embedding = combined_embedding / combined_embedding.norm(dim=-1, keepdim=True)
            combined_embedding = combined_embedding.cpu().numpy()
        else:
            combined_embedding = image_embedding.cpu().numpy()

    # Calculate similarities
    similarities = np.dot(all_embeddings, combined_embedding.T).flatten()

    # Get top-k indices
    top_k_indices = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in top_k_indices:
        results.append({
            'image_id': all_image_ids[idx],
            'image_path': all_image_paths[idx],
            'similarity_score': float(similarities[idx])
        })

    return results


# ============================================
# TEST THE IMPROVED RETRIEVAL
# ============================================

print("\n" + "=" * 60)
print("TESTING IMPROVED RETRIEVAL")
print("=" * 60)

# Reload model with improved weights
model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
model.to(device)
state_dict = torch.load('/content/fashion-iq/OUTPUT/openclip_model.pt', map_location=device)
model.load_state_dict(state_dict)
model.eval()

tokenizer = open_clip.get_tokenizer("ViT-B-32")

IMAGES_DIR = '/content/fashion-iq-metadata/image_url/downloaded_images_short'

# Test image
test_image_path = os.path.join(IMAGES_DIR, os.listdir(IMAGES_DIR)[0])
print(f"Reference image: {test_image_path}")

# Test 1: Image + Text
text_modification = "is shorter and sleeveless"
print(f"\nText query: '{text_modification}'")

results = retrieve_similar_images_improved(
    query_image_path=test_image_path,
    text_query=text_modification,
    model=model,
    preprocess=preprocess,
    tokenizer=tokenizer,
    device=device,
    top_k=5,
    text_weight=0.5
)

print(f"\nTop 5 Results (Image + Text Combined):")
for i, result in enumerate(results, 1):
    score = result['similarity_score']
    status = "✓ Good" if score > 0.6 else "○ OK" if score > 0.4 else "△ Low"
    print(f"{i}. ID: {result['image_id']} | Similarity: {score:.4f} {status}")

# Test 2: Image only
print("\n" + "-" * 40)
print("Image Only Search:")

results_image_only = retrieve_similar_images_improved(
    query_image_path=test_image_path,
    text_query="",
    model=model,
    preprocess=preprocess,
    tokenizer=tokenizer,
    device=device,
    top_k=5
)

print(f"\nTop 5 Results (Image Only):")
for i, result in enumerate(results_image_only, 1):
    score = result['similarity_score']
    status = "✓ Good" if score > 0.6 else "○ OK" if score > 0.4 else "△ Low"
    print(f"{i}. ID: {result['image_id']} | Similarity: {score:.4f} {status}")

print("=" * 60)


TESTING IMPROVED RETRIEVAL
Reference image: /content/fashion-iq-metadata/image_url/downloaded_images_short/B0089I9NFS.jpg

Text query: 'is shorter and sleeveless'

Top 5 Results (Image + Text Combined):
1. ID: B0089I9NFS | Similarity: 0.7611 ✓ Good
2. ID: B004BX98O6 | Similarity: 0.5914 ○ OK
3. ID: B00385WM5A | Similarity: 0.5842 ○ OK
4. ID: B008I2VXU8 | Similarity: 0.5228 ○ OK
5. ID: B0075NK230 | Similarity: 0.5137 ○ OK

----------------------------------------
Image Only Search:

Top 5 Results (Image Only):
1. ID: B0089I9NFS | Similarity: 1.0000 ✓ Good
2. ID: B0075NK230 | Similarity: 0.7035 ✓ Good
3. ID: B00385WM5A | Similarity: 0.7012 ✓ Good
4. ID: B004BX98O6 | Similarity: 0.6784 ✓ Good
5. ID: B008I2VXU8 | Similarity: 0.6421 ✓ Good


#Streamlit

In [77]:
!pip install -q streamlit pyngrok open_clip_torch

In [78]:
%%writefile app.py

import streamlit as st
import torch
import pickle
import numpy as np
from PIL import Image
import open_clip
import os
import json

# ============================================
# PAGE CONFIGURATION
# ============================================

st.set_page_config(
    page_title="Fashion Image Search with Text Modifications",
    page_icon="👗",
    layout="wide"
)

st.title("👗 Fashion Image Search with Picture Descriptions")
st.markdown("""
**Upload a reference image and describe the changes you want!**

For example: Upload a dress and type *"make it sleeveless and blue"* to find similar items with those modifications.
""")

# ============================================
# PATHS
# ============================================
MODEL_PATH = '/content/fashion-iq/OUTPUT/openclip_model.pt'
EMBEDDINGS_PATH = '/content/fashion-iq/OUTPUT/image_embeddings.pkl'
CAPTIONS_PATH = '/content/fashion-iq/captions/cap.test_split.json'

# ============================================
# LOAD MODEL AND EMBEDDINGS (CACHED)
# ============================================

@st.cache_resource
def load_model_and_data():
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Create model architecture
    model = open_clip.create_model("ViT-B-32", pretrained="openai")

    # Load trained weights
    if os.path.exists(MODEL_PATH):
        state_dict = torch.load(MODEL_PATH, map_location=device)
        model.load_state_dict(state_dict)
        model_status = "✅ Loaded fine-tuned model weights"
    else:
        model_status = "⚠️ Using pretrained weights (fine-tuned model not found)"

    model = model.to(device)
    model.eval()

    # Get preprocessing and tokenizer
    preprocess = open_clip.image_transform(model.visual.image_size, is_train=False)
    tokenizer = open_clip.get_tokenizer("ViT-B-32")

    # Load embeddings database
    embeddings_data = None
    embeddings_status = ""
    if os.path.exists(EMBEDDINGS_PATH):
        with open(EMBEDDINGS_PATH, 'rb') as f:
            embeddings_data = pickle.load(f)
        embeddings_status = f"✅ Loaded {len(embeddings_data['image_ids'])} image embeddings"
    else:
        embeddings_status = "❌ Embeddings file not found!"

    # Load sample captions for suggestions
    sample_captions = []
    if os.path.exists(CAPTIONS_PATH):
        with open(CAPTIONS_PATH, 'r') as f:
            test_data = json.load(f)
        sample_captions = list(set([
            entry['captions'][0] for entry in test_data
            if 'captions' in entry and entry['captions']
        ]))[:20]  # Get 20 unique captions

    return model, preprocess, tokenizer, embeddings_data, sample_captions, device, model_status, embeddings_status

# Load everything
model, preprocess, tokenizer, embeddings_data, sample_captions, device, model_status, embeddings_status = load_model_and_data()

# ============================================
# RETRIEVAL FUNCTIONS
# ============================================

def retrieve_with_image_and_text(uploaded_image, text_query, text_weight=0.5, top_k=6):
    """
    Core retrieval function combining reference image with text modification.
    Implements the project's main feature.
    """
    if embeddings_data is None:
        return []

    model.eval()
    with torch.no_grad():
        # Encode reference image
        image_tensor = preprocess(uploaded_image).unsqueeze(0).to(device)
        image_embedding = model.encode_image(image_tensor)
        image_embedding = image_embedding / image_embedding.norm(dim=-1, keepdim=True)
        image_embedding = image_embedding.cpu().numpy()

        # Encode text modification if provided
        if text_query and text_query.strip():
            text_tokens = tokenizer([text_query]).to(device)
            text_embedding = model.encode_text(text_tokens)
            text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)
            text_embedding = text_embedding.cpu().numpy()

            # Combine embeddings with weighted average
            combined_embedding = (1 - text_weight) * image_embedding + text_weight * text_embedding
            combined_embedding = combined_embedding / np.linalg.norm(combined_embedding, axis=-1, keepdims=True)
        else:
            combined_embedding = image_embedding

    # Calculate similarities
    all_embeddings = embeddings_data['embeddings']
    similarities = np.dot(all_embeddings, combined_embedding.T).flatten()

    # Get top-k results
    top_k_indices = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in top_k_indices:
        results.append({
            'image_id': embeddings_data['image_ids'][idx],
            'image_path': embeddings_data['image_paths'][idx],
            'similarity': float(similarities[idx])
        })

    return results


def retrieve_text_only(text_query, top_k=6):
    """Retrieve images using only text description."""
    if embeddings_data is None or not text_query:
        return []

    model.eval()
    with torch.no_grad():
        text_tokens = tokenizer([text_query]).to(device)
        text_embedding = model.encode_text(text_tokens)
        text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)
        text_embedding = text_embedding.cpu().numpy()

    all_embeddings = embeddings_data['embeddings']
    similarities = np.dot(all_embeddings, text_embedding.T).flatten()
    top_k_indices = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in top_k_indices:
        results.append({
            'image_id': embeddings_data['image_ids'][idx],
            'image_path': embeddings_data['image_paths'][idx],
            'similarity': float(similarities[idx])
        })

    return results


def generate_description(uploaded_image, all_descriptions):
    """Generate description for an image using CLIP similarity."""
    if not all_descriptions:
        return "No descriptions available", 0.0

    text_tokens = tokenizer(all_descriptions).to(device)
    image_tensor = preprocess(uploaded_image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        text_features = model.encode_text(text_tokens)

        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        similarities = (image_features @ text_features.T).cpu().numpy().flatten()

    best_idx = np.argmax(similarities)
    return all_descriptions[best_idx], float(similarities[best_idx])

# ============================================
# SIDEBAR
# ============================================

with st.sidebar:
    st.header("⚙️ Settings")

    # Search mode selection
    search_mode = st.radio(
        "Search Mode",
        ["🖼️ + 📝 Image + Text (Recommended)", "📝 Text Only", "🖼️ Image Only"],
        help="Choose how you want to search for fashion items"
    )

    st.markdown("---")

    # Advanced settings
    st.subheader("Advanced Settings")
    top_k = st.slider("Number of results", min_value=3, max_value=12, value=6)

    if "Image + Text" in search_mode:
        text_weight = st.slider(
            "Text Influence",
            min_value=0.0,
            max_value=1.0,
            value=0.5,
            help="0 = Image only, 1 = Text only, 0.5 = Equal balance"
        )
    else:
        text_weight = 0.5

    show_scores = st.checkbox("Show similarity scores", value=True)

    st.markdown("---")

    # Status indicators
    st.subheader("📊 System Status")
    st.write(model_status)
    st.write(embeddings_status)

    st.markdown("---")

    # Example queries
    st.subheader("💡 Example Text Queries")
    st.markdown("""
    - *"is shorter and has no sleeves"*
    - *"is black instead of white"*
    - *"has longer sleeves"*
    - *"is more casual"*
    - *"has a floral pattern"*
    """)

# ============================================
# MAIN CONTENT
# ============================================

# Create two columns for input
col_upload, col_text = st.columns([1, 1])

with col_upload:
    st.subheader("📤 Reference Image")
    uploaded_file = st.file_uploader(
        "Upload a fashion image",
        type=['jpg', 'jpeg', 'png'],
        help="Upload a reference image to find similar items"
    )

with col_text:
    st.subheader("📝 Text Modification")

    # Text input for modifications
    text_query = st.text_input(
        "Describe the changes you want",
        placeholder="e.g., 'make it sleeveless and blue'",
        help="Describe how you want the results to differ from the reference image"
    )

    # Quick suggestion buttons
    if sample_captions:
        st.caption("Quick suggestions (click to use):")
        suggestion_cols = st.columns(2)
        for i, caption in enumerate(sample_captions[:6]):
            with suggestion_cols[i % 2]:
                if st.button(caption[:40] + "..." if len(caption) > 40 else caption, key=f"sug_{i}"):
                    text_query = caption

# ============================================
# SEARCH EXECUTION
# ============================================

st.markdown("---")

# Determine what to search
has_image = uploaded_file is not None
has_text = text_query and text_query.strip()

if has_image or has_text:
    # Display inputs
    if has_image:
        uploaded_image = Image.open(uploaded_file).convert("RGB")

        input_col1, input_col2 = st.columns([1, 2])

        with input_col1:
            st.subheader("🖼️ Your Reference Image")
            st.image(uploaded_image, use_container_width=True)

            # Show detected description
            if sample_captions:
                with st.spinner("Analyzing image..."):
                    desc, conf = generate_description(uploaded_image, sample_captions)
                    st.info(f"**Detected style:** {desc}")
                    if show_scores:
                        st.caption(f"Confidence: {conf:.3f}")

    # Execute search based on mode
    with st.spinner("🔍 Searching for matching items..."):
        if "Image + Text" in search_mode and has_image:
            results = retrieve_with_image_and_text(
                uploaded_image,
                text_query if has_text else "",
                text_weight=text_weight,
                top_k=top_k
            )
            search_description = f"Image + Text (weight: {text_weight:.1f})" if has_text else "Image Only"
        elif "Text Only" in search_mode and has_text:
            results = retrieve_text_only(text_query, top_k=top_k)
            search_description = "Text Only"
        elif "Image Only" in search_mode and has_image:
            results = retrieve_with_image_and_text(
                uploaded_image,
                "",
                text_weight=0,
                top_k=top_k
            )
            search_description = "Image Only"
        else:
            results = []
            search_description = "No valid input"

    # Display results
    if has_image:
        result_container = input_col2
    else:
        result_container = st

    with result_container if has_image else st.container():
        st.subheader(f"🔍 Search Results ({search_description})")

        if has_text:
            st.caption(f"Text query: *\"{text_query}\"*")

        if results:
            # Display in grid
            result_cols = st.columns(3)
            for i, result in enumerate(results):
                with result_cols[i % 3]:
                    if os.path.exists(result['image_path']):
                        result_image = Image.open(result['image_path'])
                        st.image(result_image, use_container_width=True)
                        st.caption(f"ID: {result['image_id']}")
                        if show_scores:
                            # Color-code similarity
                            sim = result['similarity']
                            if sim > 0.7:
                                st.success(f"Similarity: {sim:.3f}")
                            elif sim > 0.5:
                                st.warning(f"Similarity: {sim:.3f}")
                            else:
                                st.caption(f"Similarity: {sim:.3f}")
                    else:
                        st.warning(f"Image not found: {result['image_id']}")
        else:
            st.warning("No results found. Try a different query or image.")

else:
    # No input provided - show instructions
    st.info("👆 **Get started:** Upload an image and/or enter a text description above!")

    # Show sample images from database
    if embeddings_data is not None:
        st.markdown("### 📸 Sample Images from Database")
        st.caption("Click on any image category to see what's available:")

        sample_cols = st.columns(6)
        for i, path in enumerate(embeddings_data['image_paths'][:6]):
            with sample_cols[i]:
                if os.path.exists(path):
                    st.image(Image.open(path), use_container_width=True)
                    st.caption(f"ID: {embeddings_data['image_ids'][i][:10]}...")

# ============================================
# FOOTER
# ============================================

st.markdown("---")
st.markdown("""
<div style='text-align: center; color: gray;'>
    <p>Built with Streamlit + OpenCLIP | Fashion Image Search with Picture Descriptions</p>
    <p><small>Based on CLIP (Contrastive Language-Image Pre-training) fine-tuned on Fashion-IQ dataset</small></p>
</div>
""", unsafe_allow_html=True)

Overwriting app.py


In [79]:
# Kill any existing streamlit processes
!pkill -f streamlit

import subprocess
import time
from pyngrok import ngrok

# Set your ngrok auth token
# Get a FREE token from: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_TOKEN = "35hxGOhLKxuICq9PZpBRJ6okyaX_3ySW5H35kHnWum5cqTV5J"  # <-- REPLACE THIS!

# Authenticate ngrok
ngrok.set_auth_token(NGROK_TOKEN)

# Kill any existing ngrok tunnels
ngrok.kill()

# Start streamlit in background
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for streamlit to start
print("Starting Streamlit server...")
time.sleep(8)

# Create public URL
try:
    public_url = ngrok.connect(8501)
    print("\n" + "=" * 60)
    print("🚀 STREAMLIT APP IS RUNNING!")
    print("=" * 60)
    print(f"\n📱 Access your app at: {public_url}")
    print("\n⚠️  Keep this cell running to maintain the connection")
    print("=" * 60)
except Exception as e:
    print(f"Error: {e}")
    print("\nTrying alternative method...")

Starting Streamlit server...

🚀 STREAMLIT APP IS RUNNING!

📱 Access your app at: NgrokTunnel: "https://dorothy-unclever-transversally.ngrok-free.dev" -> "http://localhost:8501"

⚠️  Keep this cell running to maintain the connection
